# Convert `dtype`

Sometimes the pandas data types do not fit really well. This can be due to serialisation formats that do not contain type information, for example. However, sometimes you should also change the type to achieve better performance – either more manipulation possibilities or less memory requirements. In the following examples, we will make different conversions of a `Series`:

In [1]:
import pandas as pd
import numpy as np

In [2]:
s = pd.Series(np.random.randn(7))

In [3]:
s

0    1.825581
1   -0.159163
2   -0.134431
3   -2.062616
4   -0.272327
5   -0.620893
6    1.227431
dtype: float64

## Automatic conversion

[pandas.Series.convert_dtypes](https://pandas.pydata.org/docs/reference/api/pandas.Series.convert_dtypes.html) tries to convert a `Series` to a type that supports `NA`. In the case of our `Series`, the type is changed from `float64` to `Float64`:

In [4]:
s.convert_dtypes()

0    1.825581
1   -0.159163
2   -0.134431
3   -2.062616
4   -0.272327
5   -0.620893
6    1.227431
dtype: Float64

Unfortunately, however, with `convert_dtypes` I have little control over what data type is converted to. Therefore, I prefer [pandas.Series.astype](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html):

In [5]:
s.astype('Float32')

0    1.825581
1   -0.159163
2   -0.134431
3   -2.062616
4   -0.272327
5   -0.620893
6    1.227431
dtype: Float32

Using the correct type can save memory. The usual data type is 8 bytes wide, for example `int64` or `float64`. If you can use a narrower type, this will significantly reduce memory consumption, allowing you to process more data. You can use NumPy to check the limits of integer and float types:

In [6]:
np.iinfo('int64')

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [7]:
np.finfo('float32')

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [8]:
np.finfo('float64')

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

## Memory usage

To calculate the memory consumption of the `Series`, you can use [pandas.Series.nbytes](https://pandas.pydata.org/docs/reference/api/pandas.Series.nbytes.html) to determine the memory used by the data. [pandas.Series.memory_usage](https://pandas.pydata.org/docs/reference/api/pandas.Series.memory_usage.html) also records the index memory and the data type. With `deep=True` you can also determine the memory consumption at system level.

In [9]:
s.nbytes

56

In [10]:
s.astype('Float32').nbytes

35

In [11]:
s.memory_usage()

184

In [12]:
s.astype('Float32').memory_usage()

163

In [13]:
s.memory_usage(deep=True)

184

## String and category types

The [pandas.Series.astype](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html) method can also convert numeric series into strings if you pass `str`. Note the `dtype` in the following example:

In [14]:
s.astype(str)

0       1.825581032428478
1    -0.15916250984561559
2    -0.13443077673570425
3      -2.062615930501106
4     -0.2723265429005588
5     -0.6208930961339176
6      1.2274312024234426
dtype: object

In [15]:
s.astype(str).memory_usage()

184

In [16]:
s.astype(str).memory_usage(deep=True)

658

To convert to a categorical type, you can pass `'category'` as the type:

In [17]:
s.astype(str).astype('category')

0       1.825581032428478
1    -0.15916250984561559
2    -0.13443077673570425
3      -2.062615930501106
4     -0.2723265429005588
5     -0.6208930961339176
6      1.2274312024234426
dtype: category
Categories (7, object): ['-0.13443077673570425', '-0.15916250984561559', '-0.2723265429005588', '-0.6208930961339176', '-2.062615930501106', '1.2274312024234426', '1.825581032428478']

A categorical `Series` is useful for string data and can lead to large memory savings. This is because when converting to categorical data, pandas no longer uses Python strings for each value, but repeating values are not duplicated. You still have all the features of the `str` attribute, but you save a lot of memory when you have a lot of duplicate values and you increase performance because you don’t have to do as many string operations.

In [18]:
s.astype('category').memory_usage(deep=True)

491

## Ordered categories

To create ordered categories, you need to define your own [pandas.CategoricalDtype](https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html):

In [19]:
from pandas.api.types import CategoricalDtype

sorted = pd.Series(sorted(set(s)))
cat_dtype = CategoricalDtype(
    categories=sorted, ordered=True)

s.astype(cat_dtype)

/srv/jupyter/spack/var/spack/environments/python-38/.spack-env/view/lib/python3.8/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


0    1.825581
1   -0.159163
2   -0.134431
3   -2.062616
4   -0.272327
5   -0.620893
6    1.227431
dtype: category
Categories (7, float64): [-2.062616 < -0.620893 < -0.272327 < -0.159163 < -0.134431 < 1.227431 < 1.825581]

In [20]:
s.astype(cat_dtype).memory_usage(deep=True)

491

The following table lists the types you can pass to `astype`.

Data type | Description
:-------- | :----------
`str`, `'str'` | convert to Python string
`'string'` | convert to Pandas string with `pandas.NA`
`int`, `'int'`, `'int64'` | convert to NumPy `int64`
`'int32'`, `'uint32'` | convert to NumPy `int32`
`'Int64'` | convert to pandas `Int64` with `pandas.NA`
`float`, `'float'`, `'float64'` | convert to floats
`'category'` | convert to `CategoricalDtype` with `pandas.NA`

## Conversion to other data types

The [pandas.Series.to_numpy](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_numpy.html) method or the [pandas.Series.values](https://pandas.pydata.org/docs/reference/api/pandas.Series.values.html) property gives us a NumPy array of values, and [pandas.Series.to_list](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_list.html) returns a Python list of values. Why would you want to do this? pandas objects are usually much more user-friendly and the code is easier to read. Also, python lists will be much slower to process. With [pandas.Series.to_frame](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_frame.html) you can create a DataFrame with a single column, if necessary:

In [21]:
s.to_frame()

,0
0,1.825581
1,-0.159163
2,-0.134431
3,-2.062616
4,-0.272327
5,-0.620893
6,1.227431


The function [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) can also be useful to convert values in pandas to date and time.